In [ ]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
from scipy.stats import norm
from dython import nominal

sns.set()
warnings.filterwarnings('ignore')

single_plot_w, single_plot_h = 1.5 * 3.8, 3.8
ylim = (0, 10 ** 6)

In [ ]:
# baixando dados (necessário kaggle.json, aceitar termos da competição)
# !kaggle competitions download -c house-prices-advanced-regression-techniques -p 'data/house_prices'
# !unzip -o data/house_prices/house-prices-advanced-regression-techniques.zip -d 'data/house_prices'
# !rm data/house_prices/house-prices-advanced-regression-techniques.zip

data_dir = os.path.join('data', 'house_prices')

for dirname, _, filenames in os.walk(data_dir):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
df = pd.read_csv(os.path.join(data_dir, 'train.csv'))

print(df.columns)
df.head()

In [ ]:
# Procurando valores faltantes:
df.columns[df.isna().any()]

In [ ]:
fig, (ax_box, ax_hist) = plt.subplots(2, sharex=True, gridspec_kw={"height_ratios": (.15, .85)}, figsize=(10, 6))

# Boxplot
_ = sns.boxplot(x=df['SalePrice'], ax=ax_box)
ax_box.set(xlabel='')

# Histograma
_ = sns.distplot(df['SalePrice'], ax=ax_hist)  # Set norm_hist to False
ax_hist.set_ylabel('Frequency')  # Set y-axis label to indicate frequency

plt.show()
print(f"Medida de assimetria (Skewness): {df['SalePrice'].skew()}; Medida de curtose: {df['SalePrice'].kurt()}")

Assim podemos verificar que nossa distribuição é Leptocúrtica (curtose acima de 0, afunilada no pico em relação à normal) e Positivamente Assimétrica (Cauda mais longa à direita). 

Observamos também outliers, preços altos que se destoam dos demais.

Aplicamos então uma transformação logarítmica em nosso target:

In [ ]:
logSalePrice = np.log(df['SalePrice'])

fig, (ax_box, ax_hist) = plt.subplots(2, sharex=True, gridspec_kw={"height_ratios": (.15, .85)}, figsize=(10, 6))
_ = sns.boxplot(x=logSalePrice, ax=ax_box)
ax_box.set(xlabel='')
_ = sns.distplot(logSalePrice, ax=ax_hist, fit=norm)
plt.show()

print(f"Medida de assimetria (Skewness): {logSalePrice.skew()}; Medida de curtose: {logSalePrice.kurt()}")

Com isso basicamente nos aproximamos de uma distribuição Normal, o que utilizaremos na etapa de modelagem.

#### Avaliação dos Features

Se verificarmos a base de dados apenas pelo tipo em que os dados são codificados:

In [ ]:
df.dtypes.value_counts()

Mas não conseguimos na realidade concluir quais features são numéricas ou categóricas, pois a realidade da base é um pouco mais complexa, o que concluímos ao verificar a descrição das features disponível no arquivo data_description.txt.

Após analisar todas as features listadas, podemos dividi-las em:

18 features numéricos contínuos que se referem a dimensões / área (em pés / pés quadrados)

1 feature numérico contínuo de valores monetários

9 features numéricos discretos de quantidades referentes a alguma característica específica da casa

5 features numéricos discretos relacionados a datas 

22 features categóricos ordinais 

24 features categóricos nominais

#### Análise dos Features e de sua relação com o target

Nessa etapa plotamos as distribuições de cada features, bem como os gráficos de dispersão, em relação ao preço de venda, para as features numéricas, e bloxplots para as features categóricas, divididos conforme grupos citados anteriormente.

Assim, para os os features numéricos que se referem a dimensões / área:

In [ ]:
area_features = [
    "LotFrontage",
    "LotArea",
    "MasVnrArea",
    "BsmtFinSF1",
    "BsmtFinSF2",
    "BsmtUnfSF",
    "TotalBsmtSF",
    "1stFlrSF",
    "2ndFlrSF",
    "LowQualFinSF",
    "GrLivArea",
    "GarageArea",
    "WoodDeckSF",
    "OpenPorchSF",
    "EnclosedPorch",
    "3SsnPorch",
    "ScreenPorch",
    "PoolArea",
]

In [ ]:
def plot_numeric(feature_list, ylim=(0, 10 ** 6), single_plot_w=1.5 * 3.8, single_plot_h=3.8):
    m, n = len(feature_list), 2
    print(m,n)
    fig, ax = plt.subplots(m, n, figsize=(n * single_plot_w, m * single_plot_h))
    for i, feature in enumerate(feature_list):
        g = sns.distplot(df[feature], ax=ax[i][0], kde=False)
        g = sns.regplot(x=df[feature], y=df['SalePrice'], ax=ax[i][1])
        g.set(ylim=ylim)

plot_numeric(area_features)

Podemos traçar algumas conclusões com relação a estes gráficos:

Como esperado, o preço de venda possui relação aproximadamente linear com a área útil total acima do nível do solo (GrLivArea), mas com considerável heteroscedasticidade. Também podemos observar alguns outliers (valores para áreas acima de 4000 pés quadrados), mas que segundo [ 1 ] não representam observações incorretas, e sim valores não usuais de venda;

Outros features de área também possuem relação aproximadamente linear com o preço de venda, por exemplo, área total do porão;

Moradores de Ames parecem não ser muito fãs de piscinas...

Observação: notamos que, se utilizarmos a transformação logarítmica em SalePrices, reduzimos também heteroscedasticidade em relação às medidas de área. Por exemplo:

In [ ]:
fig, ax = plt.subplots(figsize=(single_plot_w, single_plot_h))
g = sns.regplot(x=df['GrLivArea'], y=np.log(df['SalePrice']))

Com relação a valores monetários, possuímos apenas um feature, correspondente ao valor de itens adicionais que uma casa possuí, o qual plotamos por:

In [ ]:
m, n = 1, 2
fig, (ax1, ax2) = plt.subplots(m, n, figsize=(n * single_plot_w, m * single_plot_h))
g1 = sns.distplot(df['MiscVal'], ax=ax1, kde=False)
g2 = sns.regplot(x=df['MiscVal'], y=df['SalePrice'], ax=ax2)
_ = g2.set(ylim=ylim)

Aparentemente, os valores de itens adicionais, individualmente, parecem pouco informativos sobre o valor do imóvel.

Para os features numéricos discretos de quantidades de itens específicos da casa:

In [ ]:
def plot_categorical(feature_list, single_plot_w=1.5 * 3.8, single_plot_h=3.8):
    m, n = len(feature_list), 2
    fig, ax = plt.subplots(m, n, figsize=(n * single_plot_w, m * single_plot_h))
    for i, feature in enumerate(feature_list):
        g = sns.countplot(x=df[feature], ax=ax[i][0])
        g = sns.boxplot(x=df[feature], y=df['SalePrice'], ax=ax[i][1])       
        g.set_yticklabels(['{:,.0f}'.format(y) + 'K' for y in g.get_yticks()/1000])

discrete_features = ['BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageCars']
plot_categorical(discrete_features)

Podemos verificar que algumas quantidades possuem boa correlação com o preço do imóvel (ex: número de banheiros, quando diferente de zero), apesar de, em alguns casos, a relação para determinados valores não ser óbvia, possivelmente pela interação com outra variável ou pelo tamanho da amostra. Por exemplo, o número de quartos acima do nível do solo, em média, é correlacionado com o preço do imóvel quando de dois a quatro quartos, mas tal correlação não se mantém para o próximo quarto. Porém, o número de amostras para 1 ou 5 quartos é muito menor em relação às de 2 a 4 quartos. Algo semelhante ocorre para cômodos totais acima do nível do solo: temos uma relação quase linear em média, até chegarmos a 12 ou mais cômodos, quando também o número de amostras é bem menor. Para garagens, o mesmo, até a terceira garagem, mas uma quarta "reduz" o preço do imóvel (mas neste caso quase não há observações). Outro fato interessante é que ter duas cozinhas não parece ser um bom negócio...

Para os features relacionados a datas:

In [ ]:
date_features = ['YearBuilt', 'YearRemodAdd', 'GarageYrBlt', 'MoSold', 'YrSold']

m, n = 3, 1

fig, ax = plt.subplots(m, n, figsize=(n * single_plot_w * 4, m * single_plot_h * 2))
for feature, subplot in zip(date_features[:3], ax.flatten()):
    g = sns.countplot(x=df[feature], ax=subplot)
    g.set_xticklabels(g.get_xticklabels(), rotation=90)

In [ ]:
fig, ax = plt.subplots(m, n, figsize=(n * single_plot_w * 4, m * single_plot_h * 2))
for feature, subplot in zip(date_features[:3], ax.flatten()):
    g = sns.boxplot(x=df[feature], y=df['SalePrice'], ax=subplot)
    g.set_xticklabels(g.get_xticklabels(), rotation=90)
    g.set_yticklabels(['{:,.0f}'.format(y) + 'K' for y in g.get_yticks()/1000])

In [ ]:
plot_categorical(date_features[3:])

Como possíveis conclusões:

Todas as casas construídas antes de 1950, que não tiveram reforma após isso, estão com ano da última reforma em 1950 (lembrando que, se não houve reforma, esse ano deveria indicar o ano de construção, conforme descrição da documentação).

É possível notar um vácuo de construções na década de 80...

Construções recentes tendem a sinalizar, em média, um maior preço de venda, conforme seria realmente esperado.

O ano de venda não parece ser muito informativo sobre o preço de venda.

O mercado é mais aquecido no verão americano.

Já para os features ordinais categóricos, antes de plotar, estabelecemos a ordenação adequada para cada um (infelizmente, cada feature, em geral, possui uma ordenação diferente do outro), com exceção para alguns de qualidade:

In [ ]:
category_ordinal_features = ['LotShape', 'Utilities', 'LandSlope', 'OverallQual', 'OverallCond', 'ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'BsmtFinType1', 'BsmtFinType2', 'BsmtExposure', 'HeatingQC', 'KitchenQual', 'FireplaceQu', 'GarageFinish', 'GarageQual', 'GarageCond', 'PoolQC', 'Fence']
def plot_categorial_ordered(feature_list, order_list, single_plot_w=1.5 * 3.8, single_plot_h=3.8):
    m, n = len(feature_list), 2
    fig, ax = plt.subplots(m, n, figsize=(n * single_plot_w, m * single_plot_h))
    for i, feature in enumerate(feature_list):
        if order_list[i] != "":
            g = sns.countplot(x=df[feature], ax=ax[i][0], order=order_list[i])
            g = sns.boxplot(x=df[feature], y=df['SalePrice'], ax=ax[i][1], order=order_list[i])
            _ = g.set_yticklabels(['{:,.0f}'.format(y) + 'K' for y in g.get_yticks()/1000])
        else:
            g = sns.countplot(x=df[feature], ax=ax[i][0])
            g = sns.boxplot(x=df[feature], y=df['SalePrice'], ax=ax[i][1])
            _ = g.set_yticklabels(['{:,.0f}'.format(y) + 'K' for y in g.get_yticks()/1000])

order_list = []
for feature in category_ordinal_features:
    if feature in ['ExterQual', 'PoolQC']:
        order_list.append(["Fa", "TA", "Gd", "Ex"])
    elif feature in ['BsmtQual', 'BsmtCond', 'HeatingQC', 'KitchenQual', 'GarageQual', 'GarageCond', 'FireplaceQu', 'ExterCond']:
        order_list.append(["Po", "Fa", "TA", "Gd", "Ex"])
    elif feature in ['BsmtExposure']:
        order_list.append(["No", "Mn", "Av", "Gd"])
    elif feature in ['BsmtFinType1', 'BsmtFinType2']:
        order_list.append(["Unf", "LwQ", "Rec", "BLQ", "ALQ", "GLQ"])
    elif feature in ['GarageFinish']:
        order_list.append(["Unf", "RFn", "Fin"])
    elif feature in ['PavedDrive']:
        order_list.append(["N", "P", "Y"])
    elif feature in ['Fence']:
        order_list.append(["MnWw", "GdWo", "MnPrv", "GdPrv"])
    else:
        order_list.append("")

plot_categorial_ordered(category_ordinal_features, order_list)

Algumas conclusões:

Alguns features de qualidade possuem interessante relação (aproximadamente linear ou exponencial) com o preço médio, como condições do porão, acabamento da garagem, qualidade geral da casa, qualidade do porão, condições do porão, qualidade do exterior e qualidade da cozinha, mas em geral com visível heteroscedasticidade.

Uma surpresa é o feature de condições gerais da casa, que apresenta os maiores preços médios de venda para valores iguais a 5 ou 9, (e sem diferenças significativas entre 6 e 8). Notar que a maioria das observações se concentra no valor 5.

Algumas medidas de qualidade / condições são poucas informativas diretamente sobre preços para faixas de classificação de mais de um valor (ex: cercas: só há diferenciação para a última medida de qualidade, porém com poucas observações).

Alguns features possuem quase a totalidade das observações concentradas em apenas um valor: quase todas as casas tem todas as utilidades previstas na pesquisa, ou quase todas as garagens são avaliadas como TA (Average/Typical), tanto em relação a qualidade quanto as condições atuais. O mesmo ocorre na avaliação das condições do porão e nas condições externas da cada. Ou seja, as classificação de qualidade para "condições", que apresentam o campo TA (Average/Typical) como possível resposta, tendem a concentrar todas as notas neste valor, talvez por ser conveniente, em termos de avaliação de "condições", responder que o mesmo está em condições médias / típicas na avaliação do imóvel. Mas isso não ocorre na avaliação de qualidade do material (original) de qual o item é construído.

Para os features categóricos nominais, teremos:

In [ ]:
category_nominal_features = ['Neighborhood', 'Exterior1st', 'Exterior2nd', 'MSSubClass', 'MSZoning', 'Street', 'Alley', 'PavedDrive', 'LandContour', 'LotConfig', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'MasVnrType', 'Foundation', 'Heating', 'CentralAir', 'Electrical', 'GarageType', 'MiscFeature', 'Functional', 'SaleType', 'SaleCondition']

m, n = 3, 1

fig, ax = plt.subplots(m, n, figsize=(n * single_plot_w * 2, m * single_plot_h * 1.5))
for feature, subplot in zip(category_nominal_features[:3], ax.flatten()):
    g = sns.countplot(x=df[feature], ax=subplot)
    g.set_xticklabels(g.get_xticklabels(), rotation=22.5)

In [ ]:
fig, ax = plt.subplots(m, n, figsize=(n * single_plot_w * 2, m * single_plot_h * 1.5))
for feature, subplot in zip(category_nominal_features[:3], ax.flatten()):
    g = sns.boxplot(x=df[feature], y=df['SalePrice'], ax=subplot)
    g.set_yticklabels(['{:,.0f}'.format(y) + 'K' for y in g.get_yticks()/1000])
    g.set_xticklabels(g.get_xticklabels(), rotation=22.5)

In [ ]:
plot_categorical(category_nominal_features[3:])

Algumas características claramente são capazes de diferenciar diretamente o preço médio das casas. Por exemplo:

Bairros parecem ser interessante preditores de preços de casas, mas alguns bairros possuem poucas amostras.

Material de cobertura exterior da casa, se dos tipos Metal Siding e Wood Siding, que representam razoável parte da amostra, tendem a caracterizar casas mais baratas em relação as que utilizam Vinyl Siding (que é a maioria).

O tipo da área em que se encontram são bons preditores de preço médio: casas em áreas comerciais possuem preços menores, enquanto casas em vilarejos especiais (Floating Village) em geral são mais caras as demais.

A maioria das casas possui um entre dois tipos de fundação (Brick & Tile ou Cinder Block), sendo que a variância de preços relativa a cada um é baixa e os preços médios são bem distintos entre elas.

Não possuir ar central é um bom indicativo de que a casa será pouco valorizada.

Tipo da cobertura de alvenaria, quando de pedra, indica valorização no preço na casa;

Casas com garagens não anexadas na casa tendem a ser mais baratas;

Casas com sistema elétrico atualizado (padrão) tendem a valer mais;

Casas vendidas na modalidade Partial (não estavam concluídas no momento da venda, logo podemos associar com casas novas) são vendidas a um preço em geral superior as demais, como esperado.

A maioria das casas é da categoria 20 ('MSSubClass'), que correspondem a casas de um andar apenas, construídas a partir de 1946. E MSSubClass parece ser um bom preditor para os preços.

Casas em que o acesso é por estrada pavimentada possuem maior preço médio.

Em alguns features categóricos, determinada categoria acaba sendo extremamente dominante, como no caso de uma segunda característica de proximidade a alguma via importante (Condition2), material do telhado ou sistema de aquecimento.

Análise das correlações entre os features numéricos

Uma forma interessante de verificar as correlações entre as features e entre elas e o target é por meio do mapa de calor. Assim, plotamos o mapa de calor completo das variáveis numéricas (e categóricas ordinais já representadas numericamente) por:

In [ ]:
# Filter numeric and ordinal categorical columns
numeric_columns = df.select_dtypes(include=['int64', 'float64']).columns

# Concatenate the numeric and ordinal columns
selected_columns = numeric_columns.tolist() 

# Create a new DataFrame containing only the selected columns
df_selected = df[selected_columns]

# Calculate the correlation matrix for the selected columns
corr_matrix = df_selected.drop(['Id', 'MSSubClass'], axis=1).corr()

# Create the heatmap
fig, ax = plt.subplots(figsize=(10, 8))
_ = sns.heatmap(corr_matrix, annot=True, fmt='.1f', vmin=-1, vmax=1, center=0, annot_kws={"size": 5})
ax.tick_params(axis='both', which='major', labelsize=7)

plt.show()

In [ ]:

threshold = 0.5
corr_matrix = df_selected[corr_matrix[corr_matrix['SalePrice'] > threshold]['SalePrice'].index.tolist()].corr()

fig, ax = plt.subplots(figsize=(10, 8))
_ = sns.heatmap(corr_matrix, annot=True, fmt='.2f', vmin=-1, vmax=1, center=0)
plt.show()


Algumas conclusões:

Os features com as maiores correlações com o preço de venda são qualidade geral e área útil acima do solo, mas também merecem atenção área do porão, número de banheiros, área / capacidade da garagem, ano de construção, tipo de cobertura da alvenaria e quantidade de quartos, como é razoável de se supor se imaginamos o que procuramos em uma casa. Um fato curioso é a correlação entre o número de lareiras e o preço do imóvel. Já a correlação entre a área do terreno e a área útil acima do solo é apenas na ordem de 0.3.

Assim como não são correlacionados com o preço, como vimos anteriormente, algumas features quantitativas relacionadas a varandas, piscina, valor de itens diversos, mês e ano da venda também possuem baixíssima ou nenhuma correlação com os demais features ('3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MsSold', 'YrSold'). O mesmo vale para 'BsmtFinSF2' (segunda área acabada do porão), 'LowQualFinSF' (área útil de baixa qualidade), 'BsmtHalfBath' (Lavabos)

Notamos alta correlação entre alguns pares de features, em que o par possui capacidade similar de previsão de valor de venda, indicativo de multicolinearidade. São exemplos a área da garagem e sua capacidade, ano da construção da casa e ano de construção da garagem, área total útil e número de cômodos acima do solo, área do primeiro andar e área do porão (visto que para estes pares de features, em geral, uma é consequência natural da outra).

A condição geral da casa não é correlacionada com o preço devido ao fato que já apontamos (maioria das observações concentrada na nota 5, e que apresentam maior preço médio que as notas vizinhas, gerando um padrão não linear). Porém a condição geral é negativamente correlacionada com o ano da construção, o que seria um fato interessante a se supor.

Apenas por curiosidade, quanto mais nova a casa, menor são as áreas destinadas a varandas fechadas...

Porém, o mapa de calor só é capaz de indicar multicolinearidades entre duas features, e não entre combinações lineares. Assim, como identificar multicolinaridade perfeita não presente no mapa de calor, se existir? Com base nas features disponíveis, é de se supor que:

BsmtFinSF1 + BsmtFinSF2 + BsmtUnfSF = TotalBsmtSF (área total do porão é igual as áreas de partes específicas deste presentes na base)

1stFlrSF + 2ndFlrSF + LowQualFinSF = GrLivArea (área total acima do solo é igual a soma da área dos andares e da área declarada como não acabada, se supormos que, para casas com mais de dois andares, as áreas dos andares superiores são imputadas na área do segundo andar...)

Podemos mostrar que sim, isto ocorre, por:

In [ ]:
print(df['TotalBsmtSF'].corr(df['BsmtFinSF1'] + df['BsmtFinSF2'] + df['BsmtUnfSF']))
print(df['GrLivArea'].corr(df['1stFlrSF'] + df['2ndFlrSF'] + df['LowQualFinSF']))

Um gráfico interessante é o de dispersão entre as features, conforme apresentado abaixo (excluindo-se outliers em que área útil acima do solo é maior que 4000 pés quadrados, apenas para melhorar a visualização das relações):

In [ ]:
drop_list=['Id', 'MSSubClass', 'BsmtFinSF2', 'BsmtUnfSF', '2ndFlrSF', 'LowQualFinSF', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold', 'BsmtFinSF2', 'LowQualFinSF', 'BsmtHalfBath', 'KitchenAbvGr']
g = sns.pairplot(df[df['GrLivArea'] < 4000].drop(drop_list, axis=1))

E para ilustrar, o mesmo plot para algumas das features numéricas com maior correlação com o preço de venda:

In [ ]:
n = 6
cols = corr_matrix.drop(['GarageCars']).nlargest(n, 'SalePrice').index.tolist()
if 'GrLivArea' in cols:
    sns.pairplot(df[df['GrLivArea'] < 4000][cols]) #, size=2.5);

Com relação a esses gráficos, alguns fatos interessantes:

Casas classificadas como de melhor qualidade tendem a ser as de maior área útil acima do solo e com porões e garagens maiores;
Fica visível como a área do porão é geralmente limitada pela área do primeiro andar (o que é esperado, claro), e há alguns raros curiosos outliers em que a área do porão é maior que a área útil do primeiro piso.

Análise das associações entre os features categóricos
Para a análise de associações entre os features categóricos e entre esses e o preço de venda utilizamos a biblioteca dython de Shaked Zychlinski. Basicamente a função associations() gera o mapa de calor em função do coeficiente de incerteza de Theil's U, que é assimétrico (x ~ y ≠
y ~ x), o que permite explorar relações interessantes entre as features nominais, bem como a razão de correlação entre as features categóricas e 'SalePrices' (mais detalhes sobre esse tipo de abordagem pode ser verificado em um breve post de Zychlinski no towardsdatascience.com).

Assim, instalamos a classe nominal da biblioteca dython para a geração do mapa de calor para variáveis categóricas ordinais que não estão codificadas numericamente:

In [ ]:
nominal.associations(df[category_ordinal_features + ['SalePrice']].drop(["OverallQual", "OverallCond"], axis=1), fmt='.1f', nom_nom_assoc='theil', figsize=(12, 9));

Podemos tirar algumas conclusões:

Qualidade Externa, do Porão e da Cozinha são os itens mais associados com o valor do imóvel;
O acabamento da garagem diz muito sobre sua qualidade e condições, mas o contrário não é verdade;
Apenas como curiosidade, quem se preocupa com a qualidade da cozinha também se preocupa com a qualidade externa, mas não muito com a do porão... E a qualidade da cerca nos diz alguma coisa sobre a qualidade da piscina, mas o inverso não é verdade, bem como o acabamento da garagem diz muito sobre sua qualidade e condições, mas o inverso não é verdade...

E para a geração do mapa de calor para variáveis categóricas nominais:

In [ ]:
nominal.associations(df[category_nominal_features + ['SalePrice']], nom_nom_assoc='theil', figsize=(15, 12), fmt='.1f', cbar=False)

Podemos verificar que alguns features possuem associações significativas em ambas as direções, por exemplo:

Bairro e a Classificação do tipo da casa;

Tipo e Estilo da Casa com Classificação do tipo da casa, sendo que o Tipo e Classificação podem ser até ser redundantes de certa forma;

Condição de Venda e Tipo de Venda;

Alguns features possuem associação mais significativa em uma direção, mas não na outra:

O bairro pode explicar parte do tipo de classificação da zona em que se encontra a casa (comercial, zona agrícola, residencial), mas pelo tipo de zoneamento não é possível identificar o bairro;

O bairro permite prever parte da fundação que será utilizada, mas o inverso não é verdade;

O estilo do telhado ('RoofStyle') permite prever o seu material ('RoofMatl'), mas o inverso não é verdade;

Alguns features aparentam ser bons preditores para o preço de venda (Tipo de Fundação, Tipo da Garagem, Tipo Cobertura da Alvenaria ('MasVnrType'), Cobertura Exterior da Casa ('Exterior1st e 2nd')).

Itens diversos ('MscFeature'), 2o tipo de condições de proximidade ('Condition2') e tipo da rua ('Street') não têm poder significativo de explicar alguma feature ou o target.

Podemos também gerar um mega mapa com todas as associações entre features categóricas ordinais e nominais, incluindo "OverallQual", "OverallCond", que são representadas numericamente, por:

In [ ]:
nominal.associations(df[category_ordinal_features + category_nominal_features], nom_nom_assoc='theil', fmt='.1f', figsize=(30 * .8, 24 * .8), cbar=False)

Conclusões:

A qualidade geral da casa se associa com várias features categóricas nominais, como Tipo da Cobertura da Alvenaria, Tipo da Fundação e Tipo da Garagem, mas principalmente com o bairro em que se localiza (pelo bairro, você pode esperar qual a qualidade da casa que vai encontrar);

O bairro explica muito das características da casa;

Além dessa, não há muita associação entre as features categóricas ordinais e nominais, com exceção do tipo de garagem e sua avaliação de qualidade / condições / acabamento.